In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Task 2: Stopping to pay Premium

In [2]:
def Qx(gender,x,t,param =[]):
# This is our default mortality
    if gender == 0:
        a =[2.34544649e+01,8.70547812e-02,7.50884047e-05,-1.67917935e-02]
    else:
        a =[2.66163571e+01,8.60317509e-02,2.56738012e-04,-1.91632675e-02]
    return(np.exp(a[0]+(a[1]+a[2]*x)*x+a[3]*t))

In [3]:
x = 80
i = 0.02
v = 1/(1+i)
N=10

In [9]:
def Diff_Comm(x, n, v, specification, k=0, l0=1e5):
    """
    This function computes the differences in G_{x+k} - G_{x+n} for G in {M,N}. Moreover
    for G in {C,D} it computes G_{x+n}.
    Inputs:
    x : int : age at inception
    n : int : term of the insurance
    v : float : discount factor
    specification : str : 'M', 'N', 'C', 'D' for the
        respective commutation function
    k : int : deferment period (default 0)
    l0 : int : initial number of lives (default 100,000)
    Returns:
    result : float : value of the commutation function difference   

    """
    lx = np.zeros(n+1)
    dx = np.zeros(n+1)

    # for t in range(0,n+1):
    #     qx = Qx(gender=0, x=x+t, t=2020)
    #     if t==0:
    #         lx[t] = l0
    #         dx[t] = lx[t] * qx
    #     else:
    #         lx[t] = lx[t-1] - dx[t-1]
    #         # lx[t] = lx[t-1] * (1 - qx)
    #         dx[t] = lx[t] * qx
    lx[0] = l0        
    for t in range(0,n):
        qx = Qx(0,x+t,2020)
        lx[t+1] = lx[t]*(1 - qx)
        dx[t] = lx[t] * qx
    result = 0.0
    
    if specification =='M':
        for t in range(k,n):
            result += v **(x+t+1) * dx[t]
    elif specification =='N':
        for t in range(k,n):
            result += v**(x+t) * lx[t]
    elif specification =='C':
        result += v**(x+n+1) * dx[-1]
    elif specification =='D':
        result += v**(x+n) * lx[-1]
    else:   
        raise ValueError("Specification must be valid letter")
    return(result)

#### Subtask 2.1

In [10]:
P = 1e5 * (Diff_Comm(x=80, n=N, v=v, specification='M') + Diff_Comm(x=80, n=N, v=v, specification='D')) / Diff_Comm(x=80, n=N, v=v, specification='N')
print("The annual premium at inception is for age 80: ", P)

The annual premium at inception is for age 80:  12302.976099214855


#### Subtask 2.2

In [6]:
def k_V_x(x, n, k, v, premium, C):
    """
    This function computes the k-year temporary insurance value at age x
    Inputs:
    x : int : age at inception
    n : int : term of the insurance
    k : int : deferment period
    v : float : discount factor
    specification : str : 'M', 'N', 'C', 'D' for the
        respective commutation function
    C : float : sum insured
    Returns:
    result : float : value of the k-year temporary insurance at age x   

    """
    term1 = C*(Diff_Comm(x, n, v, specification='M', k=k) + Diff_Comm(x, n, v, specification='D', k=0))
    term2 = premium * Diff_Comm(x, n, v, specification='N', k=k)
    result = (term1 - term2) / Diff_Comm(x, k, v, specification='D', k=0)
    return result


def k_TildeL_x(x, n, k, v, premium, C):
    fac1 = k_V_x(x, n, k, v, premium, C)
    fac2 = Diff_Comm(x,k,v,specification='D', k=0) / (Diff_Comm(x,n,v,specification='M', k=k) + Diff_Comm(x,n,v,specification='D',k=0))
    return fac1 * fac2

In [7]:
# print(k_V_x(x=80, n=10, k=1, v=v, premium=P, C=1e5))
print('The benefit L~ after one premium stop is: ', k_TildeL_x(x=80, n=10, k=1, v=v, premium=P, C=1e5))

The benefit L~ after one premium stop is:  9228.773500783695


#### Subtask 2.3

In [52]:
for k in range(1, N+1):
    print('Benefit level L~ at stopping time k=', k, 'is ', k_TildeL_x(x=80, n=10, k=k, v=v, premium=P, C=1e5), 'and mathematical reserve is : ', k_V_x(x=80, n=10, k=k, v=v, premium=P, C=1e5))
    # print( k_TildeL_x(x=80, n=10, k=k, v=v, premium=P, C=1e5))

Benefit level L~ at stopping time k= 1 is  9228.773500783695 and mathematical reserve is :  8062.412800927408
Benefit level L~ at stopping time k= 2 is  18375.481850308755 and mathematical reserve is :  16260.208989727094
Benefit level L~ at stopping time k= 3 is  27498.514655300485 and mathematical reserve is :  24650.205699585593
Benefit level L~ at stopping time k= 4 is  36670.14539071584 and mathematical reserve is :  33308.27985147717
Benefit level L~ at stopping time k= 5 is  45980.82561053005 and mathematical reserve is :  42335.9949433317
Benefit level L~ at stopping time k= 6 is  55545.00479050228 and mathematical reserve is :  51870.0114594586
Benefit level L~ at stopping time k= 7 is  65509.06015080593 and mathematical reserve is :  62095.66956296867
Benefit level L~ at stopping time k= 8 is  76062.13670635992 and mathematical reserve is :  73266.93563725812
Benefit level L~ at stopping time k= 9 is  87450.96437880085 and mathematical reserve is :  85736.23958705967
Benefit 

#### Subtask 2.4

In [39]:
def A_x_n(x, n, k, v, C):
    result = C * (Diff_Comm(x, n, v, 'M', k=0, l0=1e5) + Diff_Comm(x,n,v,'D') ) / Diff_Comm(x,k,v,'D')
    return result

#### Subtask 2.4


We compute the benefits through the following way:
$$ \mathbb{E}(\text{benefits}) = v\cdot q_x \cdot L + v\cdot p_x \cdot \tilde{L}(t=1) \cdot A_{x+1:\overline{9|}} $$


In [44]:
benefits = v * (Qx(0,80,2020) * 1e5 + (1-Qx(0,80,2020)) * k_TildeL_x(x=80, n=10, k=1, v=v, premium=P, C=1e5) * A_x_n(81,9,0,v,1))
print('The resulting benefits accumulate to: ', benefits, ' which coinsicde with the premium computed in substask 2.1  \n P= ', P)

The resulting benefits accumulate to:  12302.976099214835  which coinsicde with the premium computed in substask 2.1  
 P=  12302.976099214855


#### Subtask 2.5

We compute the beneftis through the following way: 
$$

In [53]:
premiums = k_V_x(x=80, n=10, k=1, v=v, premium=P, C=1e5) + P
print('E(premium) = ', premiums)
q_x1 = Qx(gender=0, x=81, t=2020)
benefits = v * q_x1 * 1e5 + v * (1-q_x1) * k_TildeL_x(x=80, n=10, k=2, v=v, premium=P, C=1e5) * A_x_n(82,8,0,v,1)
print('E(benefits) = ', benefits)

E(premium) =  20365.388900142265
E(benefits) =  20365.388900142287


## Task 3 Disability Insurance on two lives

#### Subtask 3.1

#### Subtask 3.2